In [8]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel
load_dotenv()

True

In [9]:
model1 = ChatGroq(model='llama-3.1-8b-instant')

In [10]:
model2 = ChatGroq(model='meta-llama/llama-prompt-guard-2-22m')

In [11]:
prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

In [12]:
prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

In [13]:
prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

In [14]:
parser = StrOutputParser()

In [15]:
parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

In [16]:
merge_chain = prompt3 | model1 | parser

In [17]:
chain = parallel_chain | merge_chain

In [18]:
text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""


In [19]:
result = chain.invoke({'text':text})

In [20]:
print(result)

**Support Vector Machines (SVMs)**

**Definition**: Supervised learning methods for classification, regression, and outliers detection.

**Advantages**

1. Effective in high-dimensional spaces
2. Effective with many features compared to samples
3. Memory efficient using support vectors
4. Versatile with different kernel functions

**Disadvantages**

1. Overfitting occurs with many features and high regularization term
2. No direct probability estimates, use cross-validation
3. Optimal performance with dense or sparse numpy arrays

**Optimal Performance**

- Use dense or sparse numpy arrays (float64 dtype)
- Use CSR matrix for sparse data
- Example of optimal performance: 0.0010977848432958126


In [21]:
chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<notes,quiz>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP